In [ ]:
import pandas as pd
import os
os.chdir('/Users/ronaldliu/Downloads/jiang_2023/Scripts_2')

In [2]:
import pandas as pd
import glob
import os

# Path to folder with CSVs
data_dir = 'WORK_SPACE/data/stocks_dataset/raw_data/CSMAR'
csv_files = glob.glob(os.path.join(data_dir, '*.csv'))

print(f"Found {len(csv_files)} files:")
for f in csv_files:
    print(f)

df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file, dtype={'Stkcd': str})
        df_list.append(df)
    except Exception as e:
        print(f"[ERROR] {file}: {e}")

# Check if we loaded anything
if not df_list:
    raise ValueError("No CSV files were successfully read. Check file encoding or structure.")

combined_df = pd.concat(df_list, ignore_index=True)
combined_df['Trddt'] = pd.to_datetime(combined_df['Trddt'], errors='coerce')
combined_df = combined_df.sort_values(by=['Stkcd', 'Trddt'])
deduped_df = combined_df.drop_duplicates(subset=['Stkcd', 'Trddt'], keep='first')

output_path = os.path.join(data_dir, 'deduped_all.csv')
deduped_df.to_csv(output_path, index=False)

print(f"Saved deduplicated data to: {output_path}")

Found 20 files:
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/3_TRD_Dalyr (1).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/2_TRD_Dalyr.csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/6_TRD_Dalyr (1).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/1_TRD_Dalyr.csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/7_TRD_Dalyr (1).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/7_TRD_Dalyr (6).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/4_TRD_Dalyr (1).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/7_TRD_Dalyr (7).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/5_TRD_Dalyr (1).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/5_TRD_Dalyr (2).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/7_TRD_Dalyr (4).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/7_TRD_Dalyr (5).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/4_TRD_Dalyr (2).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/5_TRD_Dalyr (3).csv
WORK_SPACE/data/stocks_dataset/raw_data/CSMAR/6_TRD_Dalyr (4).csv
WO

In [3]:
def print_stock_rows(df, stock_code):
    """
    Print rows from DataFrame corresponding to a specific stock code.

    Parameters:
        df (pd.DataFrame): The full DataFrame (e.g., deduped_df)
        stock_code (str): The stock code to filter (e.g., "000001")
    """
    result = df[df['Stkcd'] == stock_code]
    if result.empty:
        print(f"No records found for stock code: {stock_code}")
    else:
        print(result)

In [5]:
# Assume deduped_df is already created from earlier steps
print_stock_rows(deduped_df, "688459")

          Stkcd      Trddt  Trdsta  Opnprc  Hiprc  Loprc  Clsprc  Dnshrtrd  \
4640128  688459 2022-10-12       1    9.31  10.25   8.84    9.46  49718103   
4640129  688459 2022-10-13       1    9.30   9.39   9.01    9.04  31015298   
4640130  688459 2022-10-14       1    9.08   9.34   9.06    9.15  22196903   
4640131  688459 2022-10-17       1    9.14   9.53   9.11    9.36  20378790   
4640132  688459 2022-10-18       1    9.36   9.73   9.24    9.60  19997365   
...         ...        ...     ...     ...    ...    ...     ...       ...   
4640762  688459 2025-05-26       1    9.58   9.63   9.56    9.59    953564   
4640763  688459 2025-05-27       1    9.61   9.65   9.57    9.63   1063433   
4640764  688459 2025-05-28       1    9.60   9.66   9.58    9.61    981446   
4640765  688459 2025-05-29       1    9.65   9.78   9.58    9.73   2303256   
4640766  688459 2025-05-30       1    9.72   9.76   9.62    9.65   1060013   

            Dnvaltrd     Dsmvosd  ...  Adjprcnd  Markettype    

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

def visualize_stock_coverage(df, output_path='stock_coverage.png'):
    """
    Visualize the date coverage (first to last date) for each stock code.

    Parameters:
        df (pd.DataFrame): DataFrame with 'Stkcd' and 'Trddt' columns.
        output_path (str): Path to save the plot image.
    """
    # Ensure 'Trddt' is datetime
    df['Trddt'] = pd.to_datetime(df['Trddt'], errors='coerce')

    # Get first and last date per stock
    coverage = df.groupby('Stkcd')['Trddt'].agg(['min', 'max']).reset_index()
    coverage = coverage.sort_values('max').reset_index(drop=True)

    # Plot horizontal lines
    plt.figure(figsize=(12, max(6, len(coverage) / 40)))  # auto-scale height
    for i, row in coverage.iterrows():
        plt.plot([row['min'], row['max']], [i, i], lw=1)

    plt.yticks(range(len(coverage)), coverage['Stkcd'], fontsize=6)
    plt.xlabel('Date')
    plt.ylabel('Stock Code')
    plt.title('Stock Listing Coverage Over Time')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close()

    print(f"Saved visualization to {output_path}")
    return coverage

In [9]:
# Assuming your deduplicated DataFrame is called `deduped_df`
coverage_df = visualize_stock_coverage(deduped_df)

/var/folders/m2/zxp_b2wj3td7hjpl_fntxdq80000gn/T/ipykernel_14884/539774954.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Trddt'] = pd.to_datetime(df['Trddt'], errors='coerce')


Saved visualization to stock_coverage.png


In [10]:
num_unique_stocks = deduped_df['Stkcd'].nunique()
print(f"Number of unique stock codes: {num_unique_stocks}")

Number of unique stock codes: 5834


In [11]:
# Determine latest date in the dataset
last_date = deduped_df['Trddt'].max()

# Group by stock and find last trading date per stock
coverage = deduped_df.groupby('Stkcd')['Trddt'].agg(['min', 'max'])

# Consider a stock delisted if last trade was more than 180 days before the last date
threshold = last_date - pd.Timedelta(days=180)
delisted = coverage[coverage['max'] < threshold]
active = coverage[coverage['max'] >= threshold]

print(f"Total stocks        : {len(coverage)}")
print(f"Active stocks       : {len(active)}")
print(f"Delisted stocks     : {len(delisted)}")

Total stocks        : 5834
Active stocks       : 5509
Delisted stocks     : 325


In [13]:
deduped_df.columns

Index(['Stkcd', 'Trddt', 'Trdsta', 'Opnprc', 'Hiprc', 'Loprc', 'Clsprc',
       'Dnshrtrd', 'Dnvaltrd', 'Dsmvosd', 'Dsmvtll', 'Dretwd', 'Dretnd',
       'Adjprcwd', 'Adjprcnd', 'Markettype', 'Capchgdt', 'Ahshrtrd_D',
       'Ahvaltrd_D', 'PreClosePrice', 'ChangeRatio', 'LimitDown', 'LimitUp',
       'LimitStatus'],
      dtype='object')

In [14]:
deduped_df.head()

,Stkcd,Trddt,Trdsta,Opnprc,Hiprc,Loprc,Clsprc,Dnshrtrd,Dnvaltrd,Dsmvosd,...,Adjprcnd,Markettype,Capchgdt,Ahshrtrd_D,Ahvaltrd_D,PreClosePrice,ChangeRatio,LimitDown,LimitUp,LimitStatus
2876010,000001,1991-04-03,1,49.00,49.00,49.00,49.00,100,5000.0,1298500.0,...,49.00,4,1991-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2876011,000001,1991-04-04,1,48.76,48.76,48.76,48.76,300,15000.0,1292140.0,...,48.76,4,1991-04-03,NaN,NaN,49.00,-0.004898,NaN,NaN,0.0
2876012,000001,1991-04-05,1,48.52,48.52,48.52,48.52,200,10000.0,1285780.0,...,48.52,4,1991-04-03,NaN,NaN,48.76,-0.004922,NaN,NaN,0.0
2876013,000001,1991-04-06,1,48.28,48.28,48.28,48.28,700,34000.0,1279420.0,...,48.28,4,1991-04-03,NaN,NaN,48.52,-0.004946,NaN,NaN,0.0
2876014,000001,1991-04-08,1,48.04,48.04,48.04,48.04,200,10000.0,1273060.0,...,48.04,4,1991-04-03,NaN,NaN,48.28,-0.004971,NaN,NaN,0.0
